In [ ]:
!pip install transformers datasets
!pip install -U pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from pathlib import Path

### Convert Pdfs to text

In [ ]:
docs_folder = '/content/drive/MyDrive/Andre_legal_task/PDF_Docs'
docs_folder = Path(docs_folder)

In [ ]:
doc_paths = list(docs_folder.rglob('*.PDF'))
print(len(doc_paths))

450


In [ ]:
doc_paths[0].stem

'Basfar (respondent) v Wong (appellant) - [2022] IRLR 879'

In [ ]:
import pymupdf4llm
from tqdm import tqdm

In [ ]:
save_path = '/content/drive/MyDrive/Andre_legal_task/MD_Docs'
save_path = Path(save_path)

pbar = tqdm(doc_paths)
for doc in pbar:
  pbar.set_description(f'processing {doc.stem}')
  md_text = pymupdf4llm.to_markdown(doc, show_progress=False)
  # now work with the markdown text, e.g. store as a UTF8-encoded file
  with (save_path / f"{doc.stem}.md").open("ab") as f:
    f.write(md_text.encode())

In [ ]:
all_done = list(save_path.glob('*.md'))

In [ ]:
not_done = set([p.stem for p in doc_paths]) - set([p.stem for p in all_done])
len(not_done)

0

In [ ]:
len(all_done)

428

In [ ]:
len(doc_paths)

450

### There are duplicates we need to merge !
I have concatenated the pdfs with the same file name into one text file.

In [ ]:
from collections import defaultdict
k = defaultdict(int)
for p in doc_paths:
  k[p.stem]+= 1
  if k[p.stem] > 1:
    print(p.stem)

Basfar v Wong
EOG (anonymity order made) v Secretary of State for the Home Department - [2020] All ER (D) 37 (Dec)
_R (on the application of Galdikas and others) v Secretary of State for the Home Department - [2016]
R (on the application of FH) v Secretary of State for the Home Department
R (on the application of NN) v Secretary of State for the Home Department; R (on the application of
_R v Joseph and others (Anti-Slavery International intervening) - [2017] All ER (D) 100 (Feb)
R (on the application of DA and others) v Secretary of State for the Home Department
R (on the application of BVN) v Secretary of State for the Home Department
R (AM) v Secretary of State for the Home Department and another
Esslemont v The Information Commissioner
R v AGM
R v BNN
R v DS
R v BWM
R v BRP
R v Iyamu (Josephine)
R v Mohammed
R v S
R v Moussa
Minister for Justice and Equality v Harrison
R (on the application of AAA and others) v Secretary of State for the Home Department (United Nation
R v BXR - [202

### Remove markdown

In [ ]:
from bs4 import BeautifulSoup
from markdown import markdown

In [ ]:
plain_text_save_path = Path('/content/drive/MyDrive/Andre_legal_task/plain_text_Docs')

In [ ]:
pbar = tqdm(list(save_path.glob('*.md')))
for md_file in pbar:
  with md_file.open('r') as f:
    md = f.read()
    html = markdown(md)
    text = ''.join(BeautifulSoup(html).findAll(string=True))
    (plain_text_save_path / f'{md_file.stem}.txt').write_text(text, encoding='utf_8')


100%|██████████| 428/428 [00:54<00:00,  7.85it/s]


In [ ]:
len(list(plain_text_save_path.glob('*.txt')))

428

### The data seems clean enough for now !

In [1]:
!pip install transformers datasets

In [15]:
!dir="/content/drive/MyDrive/Andre_legal_task/Models/stella_en_1.5B_v5"
!wget "https://huggingface.co/dunzhang/stella_en_1.5B_v5/resolve/main/2_Dense_1024/pytorch_model.bin" -P "/content/drive/MyDrive/Andre_legal_task/Models/stella_en_1.5B_v5"

--2024-11-30 11:53:09--  https://huggingface.co/dunzhang/stella_en_1.5B_v5/resolve/main/2_Dense_1024/pytorch_model.bin?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.80, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/5b/ac/5bac2c5d8af0c44f9d83bd9f7e079262d0f6f841d2bb5ae1b068ee9c5d2c081a/a63cb45e697a404f9fc7f5cc570b30a282d3fb4987e337eb78739cb2a2ed255e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1733226789&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzIyNjc4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzViL2FjLzViYWMyYzVkOGFmMGM0NGY5ZDgzYmQ5ZjdlMDc5MjYyZDBmNmY4NDFkMmJiNWFlMWIwNjhlZTljNWQyYzA4MWEvYTYzY2I0NWU2OTd

In [2]:
import os
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import normalize

# The path of your model after cloning it
model_dir = "/content/drive/MyDrive/Andre_legal_task/Models/stella_en_1.5B_v5"

vector_dim = 1024
vector_linear_directory = f"2_Dense_{vector_dim}"
model = AutoModel.from_pretrained("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)
vector_linear = torch.nn.Linear(in_features=model.config.hidden_size, out_features=vector_dim)
vector_linear_dict = {
    k.replace("linear.", ""): v for k, v in
    torch.load(os.path.join(model_dir, "pytorch_model.bin")).items()
}
vector_linear.load_state_dict(vector_linear_dict)
vector_linear.cuda()

<ipython-input-2-7ba839781337>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(model_dir, "pytorch_model.bin")).items()


Linear(in_features=1536, out_features=1024, bias=True)

In [3]:
query_prompt = "Instruct: Given a web search query, retrieve relevant passages that answer the query.\nQuery: "

queries = [
    'are judo throws allowed in wrestling?',
    'how to become a radiology technician in michigan?'
    ]
queries = [query_prompt + query for query in queries]
# docs do not need any prompts
docs =  [
    "Since you're reading this, you are probably someone from a judo background or someone who is just wondering how judo techniques can be applied under wrestling rules. So without further ado, let's get to the question. Are Judo throws allowed in wrestling? Yes, judo throws are allowed in freestyle and folkstyle wrestling. You only need to be careful to follow the slam rules when executing judo throws. In wrestling, a slam is lifting and returning an opponent to the mat with unnecessary force.",
    "Below are the basic steps to becoming a radiologic technologist in Michigan:Earn a high school diploma. As with most careers in health care, a high school education is the first step to finding entry-level employment. Taking classes in math and science, such as anatomy, biology, chemistry, physiology, and physics, can help prepare students for their college studies and future careers.Earn an associate degree. Entry-level radiologic positions typically require at least an Associate of Applied Science. Before enrolling in one of these degree programs, students should make sure it has been properly accredited by the Joint Review Committee on Education in Radiologic Technology (JRCERT).Get licensed or certified in the state of Michigan."
]

# Embed the queries
with torch.no_grad():
    input_data = tokenizer(queries, padding="longest", truncation=True, max_length=512, return_tensors="pt")
    input_data = {k: v.cuda() for k, v in input_data.items()}
    attention_mask = input_data["attention_mask"]
    last_hidden_state = model(**input_data)[0]
    last_hidden = last_hidden_state.masked_fill(~attention_mask[..., None].bool(), 0.0)
    query_vectors = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    query_vectors = normalize(vector_linear(query_vectors).cpu().numpy())

# Embed the documents
with torch.no_grad():
    input_data = tokenizer(docs, padding="longest", truncation=True, max_length=512, return_tensors="pt")
    input_data = {k: v.cuda() for k, v in input_data.items()}
    attention_mask = input_data["attention_mask"]
    last_hidden_state = model(**input_data)[0]
    last_hidden = last_hidden_state.masked_fill(~attention_mask[..., None].bool(), 0.0)
    docs_vectors = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    docs_vectors = normalize(vector_linear(docs_vectors).cpu().numpy())

print(query_vectors.shape, docs_vectors.shape)
# (2, 1024) (2, 1024)

similarities = query_vectors @ docs_vectors.T
print(similarities)
# [[0.8178789  0.2958377 ]
#  [0.31938642 0.7853526 ]]

(2, 1024) (2, 1024)
[[0.8959176  0.18929483]
 [0.19414753 0.8942834 ]]


In [7]:
attention_mask.shape

torch.Size([2, 141])

In [5]:
last_hidden_state.shape

torch.Size([2, 141, 1536])

In [6]:
last_hidden.shape

torch.Size([2, 141, 1536])

In [8]:
attention_mask

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')